<a href="https://colab.research.google.com/github/ArthurPaes/Tidal-effect/blob/main/Cenario_Tidal_effect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import random #
import matplotlib.pyplot as plt
from copy import copy

StationBase = pd.DataFrame(columns = ["ID","X","Y","RP","Fr","B","U","VU","D","PRB","PRB_F","F","C","I","CoB","H"],) #antenas small
StationMacro = pd.DataFrame(columns = ["ID","X","Y","RP","Fr","B","U","VU","D","PRB","PRB_F","F","C","I","CoB","H"]) #antena macro
Usuarios = pd.DataFrame(columns=["ID", "X", "Y", "DR","R_DR","EB",
            "PRB","CQI","SINR","C","M",
            "ES"]) #usuários

#ID - Identidade
#X  - Posicao no eixo X
#Y  - Posicaoo no eixo Y
#RP - Potência de transmissão
#DR = 0 - Taxa de dados
#B  - Banda Total
#U  - Total de Usuários
#VU - Usuarios Conectados
#D  - Estação base conectada
#R_DR - Taxa de dados requerida
#EB = 0 - Esta��o base #Estação base na qual o user está conectado
#PRB = 0  - Total de PRBs (quantidade de canais)
#PRB_F  - Total de PRBs disponiveis
#CQI = 0  - Indicador de Qualidade do Canal
#SINR = 0 - Rela��o Sinal/Ruido
#C = false  - Usuario conectado?
#M  - Momento.
#ES = 0;  - 1 = Micro || 2 == Macro
#CoB  - Metros (Euclidiana) #raio de alcance
#H  - altura antena

**Cria os Usuarios de acordo com o Vetor de distribuição**

In [ ]:
def criarusuarios(U,X,Y,df):
  # U: quantidade de usuário a serem criados
  # X: Número de limite superior para uma função de criaçao de usuários de forma aleatória "X":random.randint(0,X)
  # Y: Número de limite superior para uma função de criaçao de usuários de forma aleatória "Y":random.randint(0,Y),
  # df: Data frame de usuários criado no primeiro bloco de código

  #Vetor de distribuição que simula o efeito de maré - Quantidade de usuários varia de acordo com os momentos
  D=[1.46, 1.12, 0.91, 0.78, 0.64, 0.48, 0.34, 0.25, 0.23, 0.29, 0.5, 0.93,
    1.42, 1.5, 1.19, 1, 1.21, 1.45, 1.39, 1.23, 1.17, 1.32, 1.57, 1.62]

  V = D/np.sqrt(np.linalg.norm(D)) #normaliza os valores para ficarem entre 0 e 1
  T=400*1024 # Taxa Requerida do usuário--> 400 Kbps
  TU =[] #array da quantidade de usuários ativos em um dado momento (1hora de 24hrs)
  userID=1 #ID inicial, será incrementado em 1 na finalização de criação de cada usuário
  for j in range(0,24):
    UN=U*V[j] #quantidade de usuários ativos em um dado momento
    TU.append(UN)
    for i in range(int(UN)): #itera sobre os users naquele momento
      df = df.append({"ID":userID,
                      "X":random.randint(0,X),
                      "Y":random.randint(0,Y),
                      "R_DR":T,"M":j},
                      ignore_index=True)
      userID+=1
    userID=1 #id de usuário é reinicializado - mesmo usuario em diferentes momentos
  return df,TU

**Cria as micros ou SmallCells**

In [ ]:
#Cria as micros ou SmallCells

def CriarSmall(S,X,Y,df):
  #S número de smal cells (antenas menores) a serem criadas
  #X Número de limite superior da posição X para uma função de criaçao de smalls
  #Y Número de limite superior da posição Y para uma função de criaçao de smalls
  #df Data frame de Small cells(StationBase nesse caso) criado no primeiro bloco de código

  SC = []
  Px = np.linspace(0,X,S)
  Py = np.linspace(0,Y,S)

  smallCellId =1
  for i in range(S):
    for j in range(S): #transforma em matriz - Duplica o número de S passado
      df = df.append({"ID":smallCellId,
                      "X":Px[i],
                      "Y":Py[j],
                      "RP":33, # dBm
                      "Fr":2.4e9, # 2.4 GHz
                      "D":True,
                      "PRB":50,
                      "PRB_F":50,
                      "B":18e6, # 18 MHz #banda total
                      "C":0,
                      "CoB":300, # Metros (Euclidiana) #raio de alcance
                      "H":16 #altura antena
                      },
                      ignore_index=True)
      smallCellId+=1

  return df



**Criar Macros**

In [ ]:
def CriarMacros(M,df):
  #quando user nao conseguir conectar na Small cell(micro), usuário é jogado para a macro
  #M: Quantidade de Macros a serem criadas
  #df: Data frame de Macros (StationMacro) criado no primeiro bloco de código

  Px = np.linspace(0,1000, num=M)
  Py = np.linspace(0,1000, num=M)

  for i in range(M):
    for j in range(M):
      df = df.append({"ID":i+1, #df = dataframe
                      "X":1000,
                      "Y":1000,
                      "RP":43, # dBm
                      "Fr":3.5e9, # 3.5 GHz
                      "D":True,
                      "PRB":100,
                      "PRB_F":100,
                      "B":18e6, # 18 MHz
                      "C":0,
                      "CoB":4000, # Metros (Euclidiana) #raio de alcance
                      "H":20
                      },
                      ignore_index=True)

  return df

**Calculo de capacidade do canal**

In [ ]:
import math
from numpy.ma.core import log10
import numpy

Pw=0

def CalculodeCanal(U,S,Small):
  #Para cada usuário, a função calcula todas as antenas.
  #U - Cada linha do dataframe de usuario, ou seja cada user
  #S - Cada linha do dataframe de small cells, ou seja cada small(antena)
  #small - Data frame de small cells(no caso, StationBase)

  #Calcula a distancia do usuario para cada antena
  D=(((U.loc['X']-S.loc['X'])**2) + ((U.loc['Y']-S.loc['Y'])**2))**0.5      #distancia de Euclides

  if(D <= S.loc["CoB"] and S.loc["D"]):

    WN = 7.4e-13
    I = 0
    D0 = 100
    Sv = 9.4
    V = 3e8
    L = V/S.loc["Fr"]
    Hr = 1.2
    Hb = S.loc["H"]
    E = 16
    Fr = S.loc["Fr"]

    #Parametros de cenario
    #tipo de terreno
    a = 3.6
    b = 0.005
    C = 20

    A = 20 * log10(4 * math.pi*D0 / L)
    Y = ( a- ( b*Hb )) + ( C/Hb )
    # Sv2 = 6*log10( Fr/2000 ) - 10.8*log10( Hr/2 )
    Lost = A + 10*Y*log10( D/D0 )+ Sv - E   # Calculo de Perda no Canal
    # Lost2 = A + 10*Y*log10( D/D0 )+ Sv2   # Calculo de Perda no Canal
    Pw = 10**((S.loc["RP"] - Lost)/10)/1000;




    for i in range(len(Small.index)):
      #Calcula interferencia intercelular
      if (Small.loc[i,'D'] and Small.loc[i,'ID']!= S.loc['ID']):

        Da = (((Small.loc[i,'X']-U.loc['X'])**2) + ((Small.loc[i,'Y']-U.loc['Y'])**2))**0.5
        LostA = Small.loc[i,'RP'] - (A + 10*Y*log10( Da/D0 )+ Sv - E)
        I = I + (10**(LostA/10))/1000

    SINR = (Pw / (WN + I))                  # Calculo de Sinal Intefer�ncia + ruido
    C = S.loc['B'] / S.loc['PRB']           #//Banda total de um PRB
    DR = ((C * math.log2(1+SINR))/1024)     #Datarate (taxa) com apenas 1 PRB sendo usado.
    CQI = round(1 + ((7/13)*(SINR+6)));     #Indicador de qualidade

  else:
    SINR = 0;
    DR = 0;
    CQI = 0;

  return DR,CQI,SINR






**Conexão de Usuarios e SmallCells**

In [ ]:

def round_int(x):
    if x in [float("-inf"),float("inf")]: return float("nan")
    return int(round(x))

def ConexaoUs(Us, Small):
  #Us - Data frame de Usuários(usuarios)
  #Small -  data frame de small cells(StationBase)

  DR = pd.DataFrame(np.zeros(  (len(Us.index), len(Small.index))  ))
  CQI= pd.DataFrame(np.zeros((len(Us.index), len(Small.index))))
  SINR= pd.DataFrame(np.zeros((len(Us.index), len(Small.index))))

  for i in range(0,len(Us.index)): #linha
   for j in range(0, len(Small.index)):  #coluna
      DR1, CQI1, SINR1 = CalculodeCanal(Us.iloc[i],Small.iloc[j],Small)
      DR.iloc[i,j]= DR1
      CQI.iloc[i,j]= CQI1
      SINR.iloc[i,j]=SINR1

# Calcula o SINR, CQI e DR (1 PRB) de cada usu�rio para cada Small

  for i in range(0,len(Us.index)):

      T = max(DR.loc[i,:])
      if (T==0):
        Us['DR'][i+Us.index[0]] =0
        Us['PRB'][i+Us.index[0]] =0
        Us['EB'][i+Us.index[0]]= 0
        Us['CQI'][i+Us.index[0]]=0
        Us['SINR'][i+Us.index[0]]=0
        Us['C'][i+Us.index[0]]=False
      else:
        IND = DR.loc[i,:].idxmax()
        PR = round_int(math.ceil(Us.loc[i+Us.index[0],'R_DR']/(T*1024)))
        if (Small.loc[IND,'PRB_F']>= PR):
          Us['DR'][i+Us.index[0]] =T*PR
          Us['PRB'][i+Us.index[0]] =PR
          Us['EB'][i+Us.index[0]]= IND + 1
          Us['ES'][i+Us.index[0]]= 1
          Us['CQI'][i+Us.index[0]]=CQI.loc[i,IND]
          Us['SINR'][i+Us.index[0]]=SINR.loc[i,IND]
          Us['C'][i+Us.index[0]]=True
          Small['PRB_F'][IND]=Small.loc[IND,'PRB_F']-PR

        else:
          DR[i,IND]=0

#conecta Usuarios
  for j in range(0,len(Small.index)):
    lista=[]
    Small['VU'] = Small['VU'].astype(object)

    for i in range(0,len(Us.index)):
      if(Us.loc[i+Us.index[0],'EB']==(j+1)):
        lista.append(Us.loc[i+Us.index[0],'ID'])
    Small.at[j,'VU']=lista
    Small.loc[j,'U']=len(lista)

  return Us,Small



**Conexão de Usuarios Com a Macro**
---



In [ ]:



def ConexaoUsMacro(Us, Macro):



  DR = pd.DataFrame(np.zeros((len(Us.index), len(Macro.index))))
  CQI= pd.DataFrame(np.zeros((len(Us.index), len(Macro.index))))
  SINR= pd.DataFrame(np.zeros((len(Us.index), len(Macro.index))))



  for i in range(0,len(Us.index)):
   for j in range(0, len(Macro.index)):


      DR1, CQI1, SINR1 = CalculodeCanal(Us.iloc[i],Macro.iloc[j],Macro)
      DR.iloc[i,j]= DR1
      CQI.iloc[i,j]= CQI1
      SINR.iloc[i,j]=SINR1

# Calcula o SINR, CQI e DR (1 PRB) de cada usu�rio para cada Macro

  for i in range(0,len(Us.index)):
    if(Us.loc[i+Us.index[0],'C']==False):

        T = max(DR.loc[i,:])

        if (T==0):
          Us['DR'][i+Us.index[0]] =0
          Us['PRB'][i+Us.index[0]] =0
          Us['EB'][i+Us.index[0]]= 0
          Us['CQI'][i+Us.index[0]]=0
          Us['SINR'][i+Us.index[0]]=0
          Us['C'][i+Us.index[0]]=False
        else:
          IND = DR.loc[i,:].idxmax()
          PR = round_int(math.ceil(Us.loc[i+Us.index[0],'R_DR']/(T*1024)))


          if (Macro.loc[IND,'PRB_F']>= PR):

            Us['DR'][i+Us.index[0]] =T*PR
            Us['PRB'][i+Us.index[0]] =PR
            Us['EB'][i+Us.index[0]]= IND +1
            Us['ES'][i+Us.index[0]]= 2
            Us['CQI'][i+Us.index[0]]=CQI.loc[i,IND]
            Us['SINR'][i+Us.index[0]]=SINR.loc[i,IND]
            Us['C'][i+Us.index[0]]=True
            Macro['PRB_F'][IND]=Macro.loc[IND,'PRB_F']-PR

          else:
           DR[i,IND]=0

#conecta Usuarios
  for j in range(0,len(Macro.index)):
    lista=[]
    Macro['VU'] = Macro['VU'].astype(object)
    for i in range(0,len(Us.index)):

      if(Us.loc[i+Us.index[0],'EB']==(j+1) and Us.loc[i+Us.index[0],'ES']==2):
        lista.append(Us.loc[i+Us.index[0],'ID'])
    Macro.at[j,'VU']=lista
    Macro.loc[j,'U']=len(lista)

  return Us,Macro

def round_int(x):
    if x in [float("-inf"),float("inf")]: return float("nan")
    return int(round(x))

**Calcular resultados**

In [ ]:
from pandas._libs import index
def CalculoResults(Us, Small):

  CalculoSummary = pd.DataFrame(columns = ["coveredUsers","uncoveredUsers","MediaDRusersGeral", "DrminimoUsers",
                                           "DrmaximoUsers", "MediaDRConectedUsers","DRminimoUsersMicro","DRmaximoUsersMicro", "MicrosLigadas","UsuariosConectados"])


#  1 coluna = Usuarios Cobertos
#  2 coluna = Usuarios Não Cobertos
#  3 coluna = Media DR Usuarios (Geral)
#  4 coluna = DR minimo Usuarios - DR minimo de todos os users (macro e micro)
#  5 coluna = DR maximo Usuarios - DR minimo de todos os users (macro e micro)
#  6 coluna = Média DR Usarios (Micro-Conectados)
#  7 coluna = DR minimo Usuarios (Micro) - DR minimo apenas de users ligados nas Micros
#  8 coluna = DR maximo Usuarios (Micro) - DR maximo apenas de users ligados nas Micros
#  9 coluna = Micros ligadas - Quantidade de antenas ligadas
# 10 coluna = Usuarios Conectados (Micro)
  U = len(Us.index)
  Saida = np.zeros(10)

  TotalDataRate=0
  MediaDataRateUsersConnectedToMicro=0
  MinMaxRateUsersConectedToMicroAndMacro= [math.inf, -math.inf]
  MinMaxRateUsersConectedToMicro= [math.inf, -math.inf]

  for i in range(len(Us.index)):
    if(Us.loc[i+Us.index[0],'C']==True):
      Saida[0] = Saida[0] + 1
    else:
      Saida[1]= Saida[1]+ 1

    TotalDataRate = TotalDataRate+Us.loc[i+Us.index[0],'DR']

    if(Us.loc[i+Us.index[0],'ES']==1):
      MediaDataRateUsersConnectedToMicro = MediaDataRateUsersConnectedToMicro+Us.loc[i+Us.index[0],'DR']

    if(Us.loc[i+Us.index[0],'DR']<MinMaxRateUsersConectedToMicroAndMacro[0]):
      MinMaxRateUsersConectedToMicroAndMacro[0]= Us.loc[i+Us.index[0],'DR']

    if(Us.loc[i+Us.index[0],'DR']>MinMaxRateUsersConectedToMicroAndMacro[1]):
      MinMaxRateUsersConectedToMicroAndMacro[1]= Us.loc[i+Us.index[0],'DR']

    if(Us.loc[i+Us.index[0],'DR']<MinMaxRateUsersConectedToMicro[0] and Us.loc[i+Us.index[0],'ES']==1):
      MinMaxRateUsersConectedToMicro[0]= Us.loc[i+Us.index[0],'DR']

    if(Us.loc[i+Us.index[0],'DR']> MinMaxRateUsersConectedToMicro[1] and Us.loc[i+Us.index[0],'ES']==1):
      MinMaxRateUsersConectedToMicro[1] = Us.loc[i+Us.index[0],'DR']

  Saida[2]= TotalDataRate/U
  Saida[3]= MinMaxRateUsersConectedToMicroAndMacro[0]
  Saida[4]= MinMaxRateUsersConectedToMicroAndMacro[1]
  Saida[5]= MediaDataRateUsersConnectedToMicro/U
  Saida[6]= MinMaxRateUsersConectedToMicro[0]
  Saida[7]= MinMaxRateUsersConectedToMicro[1]

  for i in range(len(Small.index)):
    if(Small.loc[i,'D']==True):
      Saida[8]= Saida[8]+1

  for i in range(len(Us.index)):
    if(Us.loc[i+Us.index[0],'C']==True and Us.loc[i+Us.index[0],"ES"]==1):
      Saida[9]=Saida[9]+1


  CalculoSummary = CalculoSummary.append({"coveredUsers":Saida[0],"uncoveredUsers": Saida[1],"MediaDRusersGeral":Saida[2], "DrminimoUsers":Saida[3],
                                          "DrmaximoUsers": Saida[4], "MediaDRConectedUsers": Saida[5],"DRminimoUsersMicro": Saida[6],
                                          "DRmaximoUsersMicro": Saida[7],"MicrosLigadas": Saida[8],"UsuariosConectados": Saida[9]
                                         },
                                         ignore_index=True)

  return CalculoSummary



**Iniciar cenario**


In [ ]:

def StartScenario(U,SC,MC):
  Small = []
  Macro = []
  X =[]
  Y = []

  X=2000
  Y=2000


  Us,TU = criarusuarios(U,X,Y,Usuarios)
  Small = CriarSmall(SC,X,Y,StationBase)
  Macro = CriarMacros(MC,StationMacro)

  return Us, Small, Macro, TU

**Selecionar Small ou Macro**

In [ ]:

def selectAntenna(antennasOn,User,Small,Macro):
  for i in range(len(Small.index)):
    Small.loc[i,'D']=False

  for i in range(antennasOn):
    Small.loc[i,'D']=True

  Us1,S1 = ConexaoUs(User,Small)

  Us1,M1= ConexaoUsMacro(Us1,Macro)

  Saida = CalculoResults(Us1,S1)

  return Saida, Us1, S1, M1


Desligar antenas e conectar novamente

In [ ]:
def turnOffAndReconnect(SmallFrame, necessaryAntennas, UserFrame, MacroFrame):
  copiedSmallFrame = copy(SmallFrame)
  copiedUserFrame = copy(UserFrame)
  copiedMacroFrame = copy(MacroFrame)

  #zerando parametros(menos posição) dos usuários para nova conexão
  for i in range(len(copiedUserFrame.index)):
   index = i+copiedUserFrame.index[0]
   copiedUserFrame.loc[index,'DR']= float("NaN")
   copiedUserFrame.loc[index,'EB']= float("NaN")
   copiedUserFrame.loc[index,'PRB']= float("NaN")
   copiedUserFrame.loc[index,'CQI']= float("NaN")
   copiedUserFrame.loc[index,'SINR']= float("NaN")
   copiedUserFrame.loc[index,'C']= False
   copiedUserFrame.loc[index,'ES']= float("NaN")

  #Zerando parâmetros das smalls
  for i in range(len(copiedSmallFrame.index)):
    copiedSmallFrame.loc[i,'D']=False
    copiedSmallFrame.loc[i,'U']=float("NaN")
    copiedSmallFrame.loc[i,'VU']=float("NaN")
    copiedSmallFrame.loc[i,'PRB_F']=50

  #Zerando parâmetros da macro
  for i in range(len(copiedMacroFrame.index)):
    copiedMacroFrame.loc[i,'U']=float("NaN")
    copiedMacroFrame.loc[i,'VU']=float("NaN")
    copiedMacroFrame.loc[i,'PRB_F']=50

  #Ligando somente antenas necessárias
  for i in range(len(copiedSmallFrame.index)):
    for item in necessaryAntennas:
      if(copiedSmallFrame.loc[i, 'ID'] == item['id']):
        copiedSmallFrame.loc[i,'D']=True

  #Reconectando
  Us1,S1 = ConexaoUs(copiedUserFrame,copiedSmallFrame)

  Us1,M1= ConexaoUsMacro(Us1,copiedMacroFrame)

  Saida = CalculoResults(Us1,S1)

  return Saida, Us1, S1, M1

**Média de DR e SINR**

In [ ]:
def Media(User):

  T = len(User)
  DR = User['DR'].sum()
  SINR = User['SINR'].sum()
  DRMAX = User['DR'].max()
  SINRMAX = User['DR'].min()

  DR = DR/T
  SINR = SINR/T
  return DR,DRMAX,SINR,SINRMAX


**Agrupa informações importantes(resumo) da macro e micro em um data-frame**



In [ ]:
def CalcSummary(User,Small, Macro, Moment):
  MicroSummary = pd.DataFrame(columns = ["MicroID","UsersConnected","TotalDataRate","DataRateMean", "Moment"])
  MacroSummary = pd.DataFrame(columns = ["MacroID","NOfUsersConnected","UsersConnected","TotalDataRate","DataRateMean", "Moment"])

  for i in range(len(Small)):
    TotalDataRate=0
    TotalUsers = Small.loc[i,'U']
    userIdsConnectedToSmall = list(map(int, Small['VU'][i]))
    usersConnected= User[User["ID"].isin(userIdsConnectedToSmall)] #Retorna os objetos de cada usuário que estão conectados na small
    for index, row in usersConnected.iterrows():
        TotalDataRate = TotalDataRate + row["DR"]

    MicroSummary = MicroSummary.append({"MicroID":Small.loc[i,'ID'],
                      "UsersConnected":TotalUsers,
                      "TotalDataRate":TotalDataRate,
                      "DataRateMean": TotalDataRate/TotalUsers,
                      "Moment": Moment
                      },
                      ignore_index=True)

  for i in range(len(Macro)):
    TotalDataRate=0
    TotalUsers = Macro.loc[i,'U']
    userIdsConnectedToMacro = list(map(int, Macro['VU'][i]))
    usersConnected= User[User["ID"].isin(userIdsConnectedToMacro)] #Retorna os objetos de cada usuário que estão conectados na Macro
    for index, row in usersConnected.iterrows():
        TotalDataRate = TotalDataRate + row["DR"]

    MacroSummary = MacroSummary.append({"MacroID":Macro.loc[i,'ID'],
                      "NOfUsersConnected":TotalUsers,
                      "UsersConnected": userIdsConnectedToMacro,
                      "TotalDataRate":TotalDataRate,
                      "DataRateMean": TotalDataRate/TotalUsers,
                      "Moment": Moment
                      },
                      ignore_index=True)

  return MicroSummary, MacroSummary

Ordenando pela vazão média

In [ ]:
def SortByUsers(microSummary):
  momentSmallsSortedByUsers =[]
  momentSmallsSortedInfo =[]
  momentSmallsWithoutDataRate =[]

  for index, row in microSummary.iterrows():
    if(math.isnan(row.DataRateMean)):
      #Separando as antenas sem data-rate para nao atrapalhar a ordenação
      momentSmallsWithoutDataRate.append(row)
    else:
      momentSmallsSortedByUsers.append(row)
  #Ordenando pela quantidade de usuários
  momentSmallsSortedByUsers.sort(key=lambda x: x.UsersConnected, reverse=True)
  #Separando apenas o id e o data-rate médio da antena
  for item in momentSmallsSortedByUsers:
    momentSmallsSortedInfo.append({'id': item.MicroID,'DrMean': item.DataRateMean})

  return momentSmallsSortedInfo


Calcular fatias


In [ ]:
def CalcFatiasVazao(microSummary):
  totalVazaoAgregadaSmallsMomento = 0
  allVazaoAgregada20 = []
  allVazaoAgregada40 = []
  allVazaoAgregada60 = []
  allVazaoAgregada80 = []
  allVazaoAgregada100 = []

  for index, row in microSummary.iterrows():
    if(not math.isnan(row.DataRateMean)):
     #Somando as vazões médias de cada antena para obter vazão agregada do momento
     totalVazaoAgregadaSmallsMomento += row.DataRateMean
  #Fatiando em 20%,40%,60%,80%,100%
  allVazaoAgregada20.append(totalVazaoAgregadaSmallsMomento*0.2)
  allVazaoAgregada40.append(totalVazaoAgregadaSmallsMomento*0.4)
  allVazaoAgregada60.append(totalVazaoAgregadaSmallsMomento*0.6)
  allVazaoAgregada80.append(totalVazaoAgregadaSmallsMomento*0.8)
  allVazaoAgregada100.append(totalVazaoAgregadaSmallsMomento)
  totalVazaoAgregadaSmallsMomento = 0
  #Ordenando
  #Separando apenas o id e o data-rate médio da antena
  fatiasVazao = [allVazaoAgregada20, allVazaoAgregada40,allVazaoAgregada60, allVazaoAgregada80, allVazaoAgregada100]
  print("FatiasVazao", fatiasVazao)
  return fatiasVazao

Antenas para suprir o momento


In [ ]:
def CalcNecessaryAntennas(fatiaVazaoAgregada, smallsDRArray):
  print("fatiaVazaoAgregada",fatiaVazaoAgregada)
  somaDasVazoes = 0
  antenasNecessariasParaSuprirUmMomento = []
  for small in smallsDRArray:
    #Somando as vazões das antenas
    somaDasVazoes += small['DrMean']
    #Adicionando as antenas necessarias para suprir a vazao agregada ao array
    antenasNecessariasParaSuprirUmMomento.append(small)
    #Se atingir a vazao agregada -> break
    if(somaDasVazoes >= fatiaVazaoAgregada): break
  print("antenasNecessariasParaSuprirUmMomento", antenasNecessariasParaSuprirUmMomento)
  return antenasNecessariasParaSuprirUmMomento


Restribuição de PRb's

In [ ]:
def RedistribuiPRB(Us, Small):

  for i in range(0,len(Small.index)):
      if (Small.loc[i,'U']> 0 and Small.loc[i,'PRB_F']>= 0):
          DIV = Small.loc[i,'U']
          lista = list(map(int, Small['VU'][i]))
          listauser= Us[Us["ID"].isin(lista)]
          TOT_PR = round_int(math.floor(Small.loc[i,'PRB_F']/DIV))

          for index, row in listauser.iterrows():
              Us['DR'][index] =Us['DR'][index]*TOT_PR
              Us['PRB'][index] = Us['PRB'][index] + TOT_PR
              Small['PRB_F'][i]=Small.loc[i,'PRB_F']-TOT_PR


  return Us,Small

K-MEANS


In [ ]:

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


import pandas as pd
import numpy as np

def clusterizationAndExtraction(MicroSummarySingleMoment, vazaoASerAtendida):

  MicroSummaryMoment = copy(MicroSummarySingleMoment)

  ##Removendo valores desnecessários
  MicroSummaryMoment.replace([np.inf, -np.inf], np.nan, inplace=True)
  MicroSummaryMoment.replace(np.nan, 0, inplace=True)

  MicrosBeforeNormalization =  copy(MicroSummaryMoment)

  #Recalculando
  kmeans = KMeans(n_clusters=18)
  y_predicted = kmeans.fit_predict(MicroSummaryMoment)
  MicroSummaryMoment['Label'] = y_predicted

  uniqueLabels = set(kmeans.labels_)

  dataFrames = []
  colors = ["green", "orange", "red", "blue", "pink", "purple", "yellow",'black', 'gray','white','green', 'orange', 'yellow', 'blue']

  #Separando o dataframe por label
  for label in uniqueLabels:
    dataFrames.append(MicroSummaryMoment[MicroSummaryMoment.Label==label])

  vazaoAgregadaClusters = []
  vazaoAgregadaSingleCluster = 0

  #Calculando a vazao agregada de cada cluster
  for dfIndex, df in enumerate(dataFrames):
    for index,row in dataFrames[dfIndex].iterrows():
      vazaoAgregadaSingleCluster += row.DataRateMean
    vazaoAgregadaClusters.append({'vazaoAgregada': vazaoAgregadaSingleCluster, 'label': dfIndex})
    vazaoAgregadaSingleCluster = 0

  #Ordenando os clusters por vazão agregada
  vazaoAgregadaClusters.sort(key=lambda x: x['vazaoAgregada'], reverse=True)
  print("vazaoAgregadaClusters", vazaoAgregadaClusters)

  somaDasVazoesDosclusters = 0
  clustersUsados = []


  for cluster in vazaoAgregadaClusters:
    #Verificando se a vazao dos cluster atende a fatia da vazao do cenário
    if(somaDasVazoesDosclusters >= vazaoASerAtendida[0]):
      break
    else:
      somaDasVazoesDosclusters += cluster['vazaoAgregada']
      clustersUsados.append(cluster)

  print("clustersUsados", clustersUsados)

  clustersFinais = []

  #Buscando o dataframe dos clusters usados para atender a vazao agregada
  for cluster in clustersUsados:
    clustersFinais.append(dataFrames[cluster['label']])

  print("clustersFinais", clustersFinais)


  #Separando as antenas dos clusters usados a serem ligadas
  antenasLigadas = []
  for cluster in clustersFinais:
    for index, row in cluster.iterrows():
     if(not math.isnan(row.DataRateMean)):
       antenasLigadas.append({'id': row.MicroID, 'DrMean': row.DataRateMean})

  print("antenasLigadas", antenasLigadas)

  # Elbow Method - Número ideal = 3
  krng = range(1,10)
  sse = []
  for k in krng:
    km = KMeans(n_clusters=k)
    km.fit(MicroSummaryMoment)
    sse.append(km.inertia_)

  print("sse", sse)
  plt.title("Método do Cotovelo")
  plt.xlabel("K")
  plt.ylabel("SSE")
  plt.plot(krng,sse)
  plt.legend()
  plt.show()

  return antenasLigadas




**Cenario**

In [ ]:
##iniciar o cenario
Us,Small,Macro,TU= StartScenario(50,5,1)
##Dividindo o dataframe de users por momento
momento = Us.groupby(Us.M)

# Listas para concatenar os frames de todos os momentos
#Primeira conexão
ListOfUserDataFrame =[]
ListOfSmallDataFrame =[]
ListOfMacroDataFrame = []
ListOfMicroSummary = []
ListOfMacroSummary = []
ListOfSaida=[]

#Conexão após ordenação por vazão média
ListOfUserDataFrameUserApproach =[]
ListOfSmallDataFrameUserApproach =[]
ListOfMacroDataFrameUserApproach = []
ListOfMicroSummaryUserApproach = []
ListOfMacroSummaryUserApproach = []
ListOfSaidaUserApproach=[]

#Conexão após clusterização pelo k-means
ListOfUserDataFrameKMeansApproach =[]
ListOfSmallDataFrameKMeansApproach =[]
ListOfMacroDataFrameKMeansApproach = []
ListOfMicroSummaryKMeansApproach = []
ListOfMacroSummaryKMeansApproach = []
ListOfSaidaKMeansApproach=[]


smallsNosMomentos = []

for moment in range(24):
 UserFrameInGivenMoment = momento.get_group(moment)
 print("Momento Atual --------------------------------------------------------------------------------------------------", moment+1)
 #Plotando o cenário
 plt.scatter(UserFrameInGivenMoment['X'], UserFrameInGivenMoment['Y'], marker='o', c = 'blue', label='Usuário')
 plt.scatter(Small['X'], Small['Y'], marker='o', c = 'red', label='Small Cell')
 plt.scatter(Macro['X'], Macro['Y'], marker='o', c = 'purple', label='Macro Cell')
 plt.xlabel("Metros")
 plt.ylabel("Metros")
 plt.legend()
 plt.show()

 #Calculando
 SaidaSelect, UsuarioSelect, SmallSelect, MacroSelect = selectAntenna(25,UserFrameInGivenMoment,Small,Macro)

 #Adicionando os dataframes de todos os momentos em arrays
 ListOfSmallDataFrame.append(SmallSelect)
 ListOfSaida.append(SaidaSelect)
 ListOfUserDataFrame.append(UsuarioSelect)
 ListOfMacroDataFrame.append(MacroSelect)

 #Calculando os summaries
 MicroSummary,MacroSummary = CalcSummary(UsuarioSelect,SmallSelect,MacroSelect, moment)

 #Adicionando os dataframes de todos os momentos em arrays
 ListOfMacroSummary.append(MacroSummary)
 ListOfMicroSummary.append(MicroSummary)


 #Fatiando a vazão agregada do momento
 fatiasVazao = CalcFatiasVazao(MicroSummary)

  #Fatiando em [0]: 20%, [1]: 40%, [2]: 60%,[3]: 80%,[4]: 100%
 fatiaAtual = fatiasVazao[0]

 #---------------------------------------------------KMEANS-------------------------------------------------------------------------------------------


 #Clusterização
 resultadoCluster = clusterizationAndExtraction(MicroSummary, fatiaAtual )


 #Desligar antenas que não são necessárias e reconectar
 SaidaKMeansApproachReconnect, UserKMeansApproachReconnect, SmallKMeansApproachReconnect, MacroKMeansApproachReconnect = turnOffAndReconnect(Small, resultadoCluster, UserFrameInGivenMoment, Macro)

 #Redistribuição de PRBS
 UsuarioRedis, SmallRedis = RedistribuiPRB(UserKMeansApproachReconnect,SmallKMeansApproachReconnect)

 #Calculando Summary Novamente
 MicroSummaryKMeansApproach,MacroSummaryKMeansApproach = CalcSummary(UserKMeansApproachReconnect,SmallKMeansApproachReconnect,MacroKMeansApproachReconnect, moment)
 #Adicionando os dataframes de todos os momentos em arrays APÓS RECONEXAO
 ListOfSaidaKMeansApproach.append(SaidaKMeansApproachReconnect)
 ListOfUserDataFrameKMeansApproach.append(UsuarioRedis)
 ListOfSmallDataFrameKMeansApproach.append(SmallRedis)
 ListOfMacroDataFrameKMeansApproach.append(MacroKMeansApproachReconnect)
 ListOfMacroSummaryKMeansApproach.append(MacroSummaryKMeansApproach)
 ListOfMicroSummaryKMeansApproach.append(MicroSummaryKMeansApproach)


 #-----------------------------------------------------------Ordenação por usuários ---------------------------------------------------------------------

 #Ordenando por DR
 moment0SmallsSortedByDr = SortByUsers(MicroSummary)

 #Calculando antenas necessarias
 result = CalcNecessaryAntennas(fatiaAtual, moment0SmallsSortedByDr )

 #Desligar antenas que não são necessárias e reconectar
 SaidaUserApproachReconnect, UserUserApproachReconnect, SmallUserApproachReconnect, MacroUserApproachReconnect = turnOffAndReconnect(Small, result, UserFrameInGivenMoment, Macro)

 #Redistribuição de PRBS
 UsuarioRedis, SmallRedis = RedistribuiPRB(UserUserApproachReconnect,SmallUserApproachReconnect)

 #Calculando Summary Novamente
 MicroSummaryUserApproach,MacroSummaryUserApproach = CalcSummary(UserUserApproachReconnect,SmallUserApproachReconnect,MacroUserApproachReconnect, moment)
 #Adicionando os dataframes de todos os momentos em arrays APÓS RECONEXAO
 ListOfSaidaUserApproach.append(SaidaUserApproachReconnect)
 ListOfUserDataFrameUserApproach.append(UsuarioRedis)
 ListOfSmallDataFrameUserApproach.append(SmallRedis)
 ListOfMacroDataFrameUserApproach.append(MacroUserApproachReconnect)
 ListOfMacroSummaryUserApproach.append(MacroSummaryUserApproach)
 ListOfMicroSummaryUserApproach.append(MicroSummaryUserApproach)

 #------------------------------------------------------------------------------------------------------------------------------------------------------------

 smallsNosMomentos.append(Small)

 #Zerando micros e macros para o próximo momento
 Small = []
 Macro = []
 Small = CriarSmall(5,2000,2000,StationBase)
 Macro = CriarMacros(1,StationMacro)
#end of for

#Concatenando o array de dataframe em um único
FinalSaidaDataFrame = pd.concat(ListOfSaida)
FinalUserDataFrame = pd.concat(ListOfUserDataFrame)
FinalSmallDataFrame = pd.concat(ListOfSmallDataFrame)
FinalMacroDataFrame = pd.concat(ListOfMacroDataFrame)
FinalMicroSummary = pd.concat(ListOfMicroSummary)
FinalMacroSummary = pd.concat(ListOfMacroSummary)

#Concatenando o array de dataframe APÓS RECONEXÃO baseada na VAZÃO MÉDIA em um único dataframe
FinalSaidaDataFrameUserApproach = pd.concat(ListOfSaidaUserApproach)
FinalUserDataFrameUserApproach = pd.concat(ListOfUserDataFrameUserApproach)
FinalSmallDataFrameUserApproach = pd.concat(ListOfSmallDataFrameUserApproach)
FinalMacroDataFrameUserApproach = pd.concat(ListOfMacroDataFrameUserApproach)
FinalMicroSummaryUserApproach = pd.concat(ListOfMicroSummaryUserApproach)
FinalMacroSummaryUserApproach = pd.concat(ListOfMacroSummaryUserApproach)

#Concatenando o array de dataframe APÓS RECONEXÃO baseada na CLUSTERIZAÇÃO pelo K-MEANS em um único dataframe
FinalSaidaDataFrameKMeansApproach = pd.concat(ListOfSaidaKMeansApproach)
FinalUserDataFrameKMeansApproach = pd.concat(ListOfUserDataFrameKMeansApproach)
FinalSmallDataFrameKMeansApproach = pd.concat(ListOfSmallDataFrameKMeansApproach)
FinalMacroDataFrameKMeansApproach = pd.concat(ListOfMacroDataFrameKMeansApproach)
FinalMicroSummaryKMeansApproach = pd.concat(ListOfMicroSummaryKMeansApproach)
FinalMacroSummaryKMeansApproach = pd.concat(ListOfMacroSummaryKMeansApproach)

#ORDENAÇÃO
#Ordenando Micros por média de DataRate
MicroSortedByDR = FinalMicroSummary.groupby('MicroID')['DataRateMean','UsersConnected'].sum().sort_values(by='DataRateMean',ascending=False)
atualizarColunaDataRate = MicroSortedByDR['DataRateMean']/24
MicroSortedByDR = MicroSortedByDR.replace (MicroSortedByDR['DataRateMean'].values, atualizarColunaDataRate)
#Ordenando Micros por quantidade de usuários conectados
MicroSortedByUsersConnected = FinalMicroSummary.groupby('MicroID')['DataRateMean','UsersConnected'].sum().sort_values(by='UsersConnected',ascending=False)
atualizarColunaDataRate = MicroSortedByUsersConnected['DataRateMean']/24
MicroSortedByUsersConnected = MicroSortedByUsersConnected.replace (MicroSortedByUsersConnected['DataRateMean'].values, atualizarColunaDataRate)


FinalSaidaDataFrame.to_excel("Saida-CalculoResults.xlsx")
FinalUserDataFrame.to_excel("User.xlsx")
FinalSmallDataFrame.to_excel("Small.xlsx")
FinalMacroDataFrame.to_excel("Macro.xlsx")
FinalMicroSummary.to_excel("MicroSummary.xlsx")
FinalMacroSummary.to_excel("MacroSummary.xlsx")

FinalSaidaDataFrameUserApproach.to_excel("Saida-CalculoResultsUserApproach.xlsx")
FinalUserDataFrameUserApproach.to_excel("UserUserApproach.xlsx")
FinalSmallDataFrameUserApproach.to_excel("SmallUserApproach.xlsx")
FinalMacroDataFrameUserApproach.to_excel("MacroUserApproach.xlsx")
FinalMicroSummaryUserApproach.to_excel("MicroSummaryUserApproach.xlsx")
FinalMacroSummaryUserApproach.to_excel("MacroSummaryUserApproach.xlsx")

FinalSaidaDataFrameKMeansApproach.to_excel("Saida-CalculoResultsKMeansApproach.xlsx")
FinalUserDataFrameKMeansApproach.to_excel("UserKMeansApproach.xlsx")
FinalSmallDataFrameKMeansApproach.to_excel("SmallKMeansApproach.xlsx")
FinalMacroDataFrameKMeansApproach.to_excel("MacroKMeansApproach.xlsx")
FinalMicroSummaryKMeansApproach.to_excel("MicroSummaryKMeansApproach.xlsx")
FinalMacroSummaryKMeansApproach.to_excel("MacroSummaryKMeansApproach.xlsx")

MicroSortedByDR.to_excel("MicroSortedByDR.xlsx")
MicroSortedByUsersConnected.to_excel("MicroSortedByUsersConnected.xlsx")
